## Init and import

This depends on a new custom module in concordia
Run "pip install --editable .[dev]" in your environment to install the new build if you haven't

In [1]:
# @title Imports
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)
import datetime

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [3]:
# from concordia.language_model import huggingface_model
import huggingface_model
import importlib
importlib.reload(huggingface_model)

# ===== MODEL LIST =====
# bigscience/bloom-560m
# mistralai/Mixtral-8x7B-v0.1 (not tested yet)
# mistralai/Mistral-7B-v0.1
# (check huggingface for more)

model = huggingface_model.HuggingFaceLanguageModel(
    model_name='mistralai/Mistral-7B-v0.1',
    logits=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [7]:
measurements = measurements_lib.Measurements()

In [8]:
import custom_components.logits_query_metric as lm
import importlib
importlib.reload(lm)

context = """
You are Alice, a very smart person.
You will be given a number of statements.
You will determine if each statement is true and give a single "yes" or "no" answer to each statement.
You will NOT say anything else other than "yes" or "no".
If you doesn't know the answer, she will guess an answer.
"""

quiz_metric = lm.LogitsQueryMetric(model, 
                                   "Alice", 
                                   clock, 
                                   "./custom_components/truth_or_lie.json", 
                                   measurements=measurements, 
                                   verbose=True,
                                   query=["yes", "no"])
quiz_metric.observe(context)

/home/yumo/miniconda3/envs/concordia/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Word response: 


Alice is a very smart person.
Alice will be given a number of statements.
She will determine if each statement is true and give a single "yes" or "no" answer to each statement.
She will not say anything else other than "yes" or "no".
If she doesn't know the answer, she will guess an answer.

1 + 1 = 2 is a true statement.
Alice will answer "yes" to this statement.

1 + 1 = 3 is a false statement.
Alice will answer "no" to this statement.

1 + 1 =
To the question 1 + 1 = 2, the agent answered
 {'yes': 0.5, 'no': 0.5}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Word response: 


Alice is a very smart person.
Alice will be given a number of statements.
She will determine if each statement is true and give a single "yes" or "no" answer to each statement.
She will not say anything else other than "yes" or "no".
If she doesn't know the answer, she will guess an answer.

Red is a color.
Blue is a color.
Red is not blue.

Alice is asked if red is blue.
She does not know the answer, so she guesses "no".

Alice is asked if red is red.
To the question Red is a color, the agent answered
 {'yes': 0.5, 'no': 0.5}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Word response: 


Alice is a very smart person.
Alice will be given a number of statements.
She will determine if each statement is true and give a single "yes" or "no" answer to each statement.
She will not say anything else other than "yes" or "no".
If she doesn't know the answer, she will guess an answer.

Earth is flat.

Alice: No

Earth is round.

Alice: Yes

Earth is round.

Alice: Yes

Earth is flat.

Alice: No

Earth
To the question Earth is flat, the agent answered
 {'yes': 0.5, 'no': 0.5}
LogitQueryScores of Alice: Agent completed the quiz.


In [12]:
import importlib
import examples.custom_components.logits_query_metric as lm
importlib.reload(lm)

results = quiz_metric.get_results()
import csv
# Create a csv file with the results if not already created
if not os.path.exists('./results/'):
    os.makedirs('./results/')
with open('./results/logits.txt', 'w') as file:
    # Write Question: Response, and space after each pair
    writer = csv.writer(file)
    for result in results:
        writer.writerow(result)
    file.close()
    